In [1]:
import numpy as np
import pandas as pd
import os
from collections import OrderedDict

In [2]:
from platform import python_version
print(python_version())

3.6.6


### Exp of classifing movie genres based on their events.


In [26]:
movies = pd.read_csv('../../data/movie_metadata_cleaned.tsv', sep = '\t')

In [27]:
movies.head(2)

,Wikipedia_id,Freebase_id,Name,Release_date,Revenue,Runtime,Languages,Countries,Genres
0,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,dict_values(['English Language']),dict_values(['United States of America']),Drama
1,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,dict_values(['Norwegian Language']),dict_values(['Norway']),Drama


### Read in events data

In [28]:
event_file = [item for item in os.listdir('../../data/protag_events_v2/') if 'txt' in item]

In [29]:
d = {}
for name in event_file:
    with open(os.path.join('../../data/protag_events_v2/', name), 'r') as f:
        #only use the verbs
        verbs = []
        for line in [line.strip() for line in f.readlines()]:
            line = line.split(' ')
            verbs.append(line[1])
        if len(verbs) > 1:    
            d[int(name.split('_')[0])] = verbs

In [8]:
d = {}
for name in event_file:
    with open(os.path.join('../../data/protag_events_v2/', name), 'r') as f:
        #only use the verbs
        nlines = []
        for line in [line.strip() for line in f.readlines()]:
            line = line.split(' ')
            line[0] = 'protagonist'
            line = ' '.join([item for item in line[0:2]])
            nlines.append(line)
        if len(nlines) > 1:    
            d[int(name.split('_')[0])] = nlines

In [30]:
movies.Wikipedia_id = movies.Wikipedia_id.astype(int)

In [31]:
movies['Events'] = movies['Wikipedia_id'].map(d)

In [32]:
movies = movies.dropna(subset = ['Events'])

In [33]:
movies['Genre_code'] = movies.Genres.astype("category").cat.codes

In [34]:
dict( enumerate(movies.Genres.astype("category").cat.categories) )

{0: 'Action', 1: 'Comedy', 2: 'Drama', 3: 'Romance Film', 4: 'Thriller'}

In [35]:
movies.head(2)

,Wikipedia_id,Freebase_id,Name,Release_date,Revenue,Runtime,Languages,Countries,Genres,Events,Genre_code
10,11250635,/m/02r52hc,The Mechanical Monsters,NaN,NaN,NaN,dict_values(['English Language']),dict_values(['United States of America']),Action,"[weather-57, battle-36.4-1, require-103-1, see...",0
51,7767613,/m/026ccts,Star!,1968-07-18,4000000.0,176.0,dict_values(['English Language']),dict_values(['United States of America']),Drama,"[mix-22.1-2-1, correspond-36.1, amuse-31.1, ge...",2


In [36]:
movies.Genre_code.value_counts()

2    1733
1    1165
4     494
0     345
3     103
Name: Genre_code, dtype: int64

In [37]:
movies.to_csv('../../data/metadata_events_table.tsv', sep = '\t', index=False)